# Setup and Imports

In [1]:
import httpx
import ssl
import asyncio
from h2.config import H2Configuration
from h2.connection import H2Connection
from h2.events import SettingsAcknowledged, ResponseReceived, DataReceived, StreamEnded, WindowUpdated, ConnectionTerminated
from h2.events import PingReceived
from h2.exceptions import ProtocolError
from h2.settings import SettingCodes
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Base URL for testing
base_url = 'https://nghttp2.org'


# Basic HTTP/2 Request

In [2]:
async def basic_http2_request():
    async with httpx.AsyncClient(http2=True) as client:
        response = await client.get(base_url)
        print("Status Code:", response.status_code)
        print("Response body:", response.text[:500])  
await basic_http2_request()


Status Code: 200
Response body: 
<!DOCTYPE html>
<!--[if IEMobile 7 ]><html class="no-js iem7"><![endif]-->
<!--[if lt IE 9]><html class="no-js lte-ie8"><![endif]-->
<!--[if (gt IE 8)|(gt IEMobile 7)|!(IEMobile)|!(IE)]><!--><html class="no-js" lang="en"><!--<![endif]-->
<head>
  <meta charset="utf-8">
  <title>Nghttp2: HTTP/2 C Library - nghttp2.org</title>
  <meta name="author" content="Tatsuhiro Tsujikawa">

  
  <meta name="description" content="Nghttp2: HTTP/2 C Library Feb 16th, 2015 11:16 pm nghttp2 is an implementation 


# Data Streaming

In [3]:
async def data_streaming():
    url = 'https://nghttp2.org/httpbin/stream/20'  # Streaming endpoint
    async with httpx.AsyncClient(http2=True) as client:
        async with client.stream("GET", url) as response:
            async for line in response.aiter_lines():
                print(line)

# Run the async function
await data_streaming()

{"url": "https://nghttp2.org/httpbin/stream/20", "args": {}, "headers": {"Host": "nghttp2.org", "Accept": "*/*", "Accept-Encoding": "gzip, deflate, br", "User-Agent": "python-httpx/0.25.2"}, "origin": "2601:282:a03:41c0:80b0:6c78:e8d3:e464", "id": 0}
{"url": "https://nghttp2.org/httpbin/stream/20", "args": {}, "headers": {"Host": "nghttp2.org", "Accept": "*/*", "Accept-Encoding": "gzip, deflate, br", "User-Agent": "python-httpx/0.25.2"}, "origin": "2601:282:a03:41c0:80b0:6c78:e8d3:e464", "id": 1}
{"url": "https://nghttp2.org/httpbin/stream/20", "args": {}, "headers": {"Host": "nghttp2.org", "Accept": "*/*", "Accept-Encoding": "gzip, deflate, br", "User-Agent": "python-httpx/0.25.2"}, "origin": "2601:282:a03:41c0:80b0:6c78:e8d3:e464", "id": 2}
{"url": "https://nghttp2.org/httpbin/stream/20", "args": {}, "headers": {"Host": "nghttp2.org", "Accept": "*/*", "Accept-Encoding": "gzip, deflate, br", "User-Agent": "python-httpx/0.25.2"}, "origin": "2601:282:a03:41c0:80b0:6c78:e8d3:e464", "id":

# Header Compression

In [4]:
async def header_compression():
    url = 'https://nghttp2.org'
    async with httpx.AsyncClient(http2=True) as client:
        response = await client.get(url)
        print("Response Headers:", response.headers)

# Run the async function
await header_compression()


Response Headers: Headers({'date': 'Mon, 11 Dec 2023 02:29:39 GMT', 'content-type': 'text/html', 'last-modified': 'Fri, 27 Oct 2023 13:41:30 GMT', 'etag': '"653bbe0a-18b4"', 'accept-ranges': 'bytes', 'content-length': '6324', 'x-backend-header-rtt': '0.00154', 'strict-transport-security': 'max-age=31536000', 'server': 'nghttpx', 'alt-svc': 'h3=":443"; ma=3600, h3-29=":443"; ma=3600', 'via': '2 nghttpx', 'x-frame-options': 'SAMEORIGIN', 'x-xss-protection': '1; mode=block', 'x-content-type-options': 'nosniff'})


# Testing Concurrent Stream Multiplexing

In [5]:
async def test_stream_multiplexing():
    url = 'https://nghttp2.org/httpbin/delay/2'  # Endpoint with a delay to simulate long-running request
    async with httpx.AsyncClient(http2=True) as client:
        # Initiating multiple requests concurrently
        responses = await asyncio.gather(
            client.get(url),
            client.get(url),
            client.get(url)
        )
        for response in responses:
            print("Response from stream:", response.status_code)

# Run the async function
await test_stream_multiplexing()


Response from stream: 200
Response from stream: 200
Response from stream: 200


# Testing HTTP/2 Prioritization

In [6]:
async def test_http2_prioritization():
    url = 'https://nghttp2.org/httpbin/delay/1'  # Delayed response
    async with httpx.AsyncClient(http2=True) as client:
        # High-priority request
        high_priority = await client.get(url, headers={'Priority': 'u=1, i'})
        print("High priority response:", high_priority.status_code)

        # Low-priority request
        low_priority = await client.get(url, headers={'Priority': 'u=4, i'})
        print("Low priority response:", low_priority.status_code)

# Run the async function
await test_http2_prioritization()


High priority response: 200
Low priority response: 200


# Simulating High Traffic

In [7]:

async def send_request(client, url):
    try:
        response = await client.get(url)
        return response.status_code
    except Exception as e:
        return str(e)

async def simulate_high_traffic(url, request_count=100):
    async with httpx.AsyncClient(http2=True) as client:
        tasks = [send_request(client, url) for _ in range(request_count)]
        responses = await asyncio.gather(*tasks)
        return responses

# URL of the test server
test_url = 'https://nghttp2.org/httpbin/anything'

# Run the simulation
responses = await simulate_high_traffic(test_url, 500)  # Example: 500 requests
print("Responses:", responses)

Responses: [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200

# Checking for Common Vulnerabilities

In [8]:

common_vulnerabilities = [
    "/.git/", "/.env", "/.htaccess", "/phpinfo.php", "/config.php"
]

async def check_vulnerabilities(url):
    async with httpx.AsyncClient(http2=True) as client:
        for path in common_vulnerabilities:
            try:
                response = await client.get(url + path)
                if response.status_code == 200:
                    print(f"Potential vulnerability found at: {url + path}")
                else:
                    print(f"No issue detected at: {url + path}")
            except Exception as e:
                print(f"Error checking {url + path}: {str(e)}")

test_url = 'https://nghttp2.org'  
await check_vulnerabilities(test_url)

No issue detected at: https://nghttp2.org/.git/
No issue detected at: https://nghttp2.org/.env
No issue detected at: https://nghttp2.org/.htaccess
No issue detected at: https://nghttp2.org/phpinfo.php
No issue detected at: https://nghttp2.org/config.php


# Checking for SSL/TLS Vulnerabilities

In [9]:

async def check_security_headers(url):
    async with httpx.AsyncClient(http2=True) as client:
        response = await client.get(url)
        headers_to_check = ["Content-Security-Policy", "X-Frame-Options", "X-Content-Type-Options"]
        for header in headers_to_check:
            if header in response.headers:
                print(f"{header} is set.")
            else:
                print(f"{header} is missing.")

test_url = 'https://nghttp2.org'  
await check_security_headers(test_url)

Content-Security-Policy is missing.
X-Frame-Options is set.
X-Content-Type-Options is set.


# Rapid Reset

In [10]:

async def send_and_reset_stream(host, port, path, ssl_context):
    start_time = time.time()

    try:
        reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)

        config = H2Configuration(client_side=True)
        conn = H2Connection(config=config)
        conn.initiate_connection()
        writer.write(conn.data_to_send())

        stream_id = conn.get_next_available_stream_id()
        headers = [
            (':method', 'GET'),
            (':authority', host),
            (':scheme', 'https'),
            (':path', path),
        ]
        conn.send_headers(stream_id, headers)
        conn.end_stream(stream_id)
        writer.write(conn.data_to_send())

        # Immediately reset the stream
        conn.reset_stream(stream_id)
        writer.write(conn.data_to_send())

        # Close connection
        conn.close_connection()
        writer.write(conn.data_to_send())
        await writer.drain()
        writer.close()

        elapsed_time = time.time() - start_time
        return ("Success", elapsed_time)

    except Exception as e:
        return (str(e), time.time() - start_time)

async def rapid_reset_test(host, port, path, ssl_context, request_count):
    tasks = [send_and_reset_stream(host, port, path, ssl_context) for _ in range(request_count)]
    results = await asyncio.gather(*tasks)
    return results

def analyze_results(results):
    response_times = [time for status, time in results if status == "Success"]
    error_counts = {status: len([1 for s, _ in results if s == status]) for status, _ in results if status != "Success"}

    analysis = {
        "Total Requests": len(results),
        "Total Success": len(response_times),
        "Total Failed": len(results) - len(response_times),
        "Success Rate": len(response_times) / len(results) * 100,
        "Average Time (s)": np.mean(response_times),
        "Max Time (s)": np.max(response_times),
        "Min Time (s)": np.min(response_times),
        "Median Time (s)": np.median(response_times),
        "95th Percentile Time (s)": np.percentile(response_times, 95),
        "Error Counts": error_counts
    }
    return analysis

async def load_test(host, port, path, start=100, end=500, step=100):
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])
    for request_count in range(start, end + 1, step):
        print(f"\nTesting with {request_count} requests...")
        results = await rapid_reset_test(host, port, path, ssl_context, request_count)
        analysis = analyze_results(results)
        for key, value in analysis.items():
            print(f"{key}: {value}")

# Test URL and path
host = 'nghttp2.org'
port = 443
path = '/'

# Run the load test
await load_test(host, port, path)


Testing with 100 requests...
Total Requests: 100
Total Success: 100
Total Failed: 0
Success Rate: 100.0
Average Time (s): 0.5544549822807312
Max Time (s): 0.6804211139678955
Min Time (s): 0.38999032974243164
Median Time (s): 0.5971882343292236
95th Percentile Time (s): 0.6674982190132142
Error Counts: {}

Testing with 200 requests...
Total Requests: 200
Total Success: 200
Total Failed: 0
Success Rate: 100.0
Average Time (s): 0.7903748822212219
Max Time (s): 1.0021076202392578
Min Time (s): 0.355273962020874
Median Time (s): 0.9333958625793457
95th Percentile Time (s): 0.995127260684967
Error Counts: {}

Testing with 300 requests...
Total Requests: 300
Total Success: 300
Total Failed: 0
Success Rate: 100.0
Average Time (s): 1.1116115935643514
Max Time (s): 1.3991739749908447
Min Time (s): 0.39573144912719727
Median Time (s): 1.2825032472610474
95th Percentile Time (s): 1.3821902871131897
Error Counts: {}

Testing with 400 requests...
Total Requests: 400
Total Success: 400
Total Failed:

In [11]:
analysis_results = []

In [12]:
# Define the analyze_results function
def analyze_results(test_name, response):
    print(f"Raw Response Data: {response}")
    # Append the test name and response to the global list
    analysis_results.append({"Test Name": test_name, "Response": response})


# Individual Frame Testing

# Header Frame

In [13]:
async def test_headers_frame_and_analyze():
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443
    path = '/'

    start_time = time.time()

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)
    
    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())

    headers = [
        (':method', 'GET'),
        (':authority', host),
        (':scheme', 'https'),
        (':path', path),
        ('user-agent', 'http2-test/0.1'),
    ]
    stream_id = conn.get_next_available_stream_id()
    conn.send_headers(stream_id, headers, end_stream=True)
    writer.write(conn.data_to_send())
    await writer.drain()

    data_received = 0

    while True:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                print('Chunk of response data received:', len(event.data))
                data_received += len(event.data)
            elif isinstance(event, StreamEnded):
                print('Stream ended')
                break

    writer.close()
    await writer.wait_closed()

    end_time = time.time()

    print(f"Total response time: {end_time - start_time:.2f} seconds")
    print(f"Total data received: {data_received} bytes")
    response_data = {
        "Total response time": end_time - start_time,
        "Total data received": data_received
    }
    analyze_results("Headers Frame Test", response_data)


await test_headers_frame_and_analyze()

Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:01 GMT'), (b'content-type', b'text/html'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-18b4"'), (b'accept-ranges', b'bytes'), (b'content-length', b'6324'), (b'x-backend-header-rtt', b'0.001536'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600, h3-29=":443"; ma=3600'), (b'via', b'2 nghttpx'), (b'x-frame-options', b'SAMEORIGIN'), (b'x-xss-protection', b'1; mode=block'), (b'x-content-type-options', b'nosniff')]
Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:01 GMT'), (b'content-type', b'text/css'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-98aa"'), (b'accept-ranges', b'bytes'), (b'content-length', b'39082'), (b'x-backend-header-rtt', b'0.001445'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600

# Data Frame

In [14]:

async def test_data_frame_and_analyze():
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443
    path = '/post'  # Change to a path that accepts POST requests

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)

    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())

    # Headers for a POST request
    headers = [
        (':method', 'POST'),
        (':authority', host),
        (':scheme', 'https'),
        (':path', path),
        ('content-length', '15'),  
        ('content-type', 'application/x-www-form-urlencoded'),
    ]
    stream_id = conn.get_next_available_stream_id()
    conn.send_headers(stream_id, headers)

    # Sending the data
    data = 'Hello, world!\n'
    conn.send_data(stream_id, data.encode('utf-8'), end_stream=True)
    writer.write(conn.data_to_send())
    await writer.drain()

    start_time = time.time()
    data_received = 0

    while True:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                data_received += len(event.data)
                print('Chunk of response data received:', len(event.data))
            elif isinstance(event, StreamEnded):
                print('Stream ended')
                break

    end_time = time.time()

    writer.close()
    await writer.wait_closed()

    print(f"Total response time: {end_time - start_time:.2f} seconds")
    print(f"Total data received: {data_received} bytes")
    response_data = {
        "Total response time": end_time - start_time,
        "Total data received": data_received
    }
    analyze_results("Data Frame Test", response_data)

await test_data_frame_and_analyze()

Total response time: 10.15 seconds
Total data received: 0 bytes
Raw Response Data: {'Total response time': 10.146790742874146, 'Total data received': 0}


# Settings Frame

In [15]:

async def test_settings_frame_and_analyze():
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)

    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())

    # Sending a SETTINGS frame with custom settings (adjust as needed)
    new_settings = {
        SettingCodes.MAX_CONCURRENT_STREAMS: 10,
    }
    conn.update_settings(new_settings)
    writer.write(conn.data_to_send())
    await writer.drain()

    start_time = time.time()

    settings_acknowledged = False
    while not settings_acknowledged:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, SettingsAcknowledged):
                settings_acknowledged = True
                print("Settings acknowledged by server")
            elif isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                print('Response data:', event.data)
            elif isinstance(event, StreamEnded):
                print('Stream ended')
                break

    end_time = time.time()

    writer.close()
    await writer.wait_closed()

    print(f"Total time for settings acknowledgment: {end_time - start_time:.2f} seconds")
    response_data = {
        "Total time for settings acknowledgment": end_time - start_time
    }
    analyze_results("Settings Frame Test", response_data)

await test_settings_frame_and_analyze()

Settings acknowledged by server
Settings acknowledged by server
Total time for settings acknowledgment: 0.29 seconds
Raw Response Data: {'Total time for settings acknowledgment': 0.28667449951171875}


# Window_Updated Frame

In [16]:

async def test_window_update_frame_and_analyze():
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443
    path = '/'  

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)

    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())

    # Headers for a GET request
    headers = [
        (':method', 'GET'),
        (':authority', host),
        (':scheme', 'https'),
        (':path', path),
    ]
    stream_id = conn.get_next_available_stream_id()
    conn.send_headers(stream_id, headers, end_stream=True)
    writer.write(conn.data_to_send())
    await writer.drain()

    # Send a WINDOW_UPDATE frame 
    conn.increment_flow_control_window(increment=1024, stream_id=stream_id)
    writer.write(conn.data_to_send())
    await writer.drain()

    start_time = time.time()

    while True:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, WindowUpdated):
                print("Window update received, stream_id:", event.stream_id)
            elif isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                print('Response data:', event.data)
            elif isinstance(event, StreamEnded):
                print('Stream ended')
                break

    end_time = time.time()

    writer.close()
    await writer.wait_closed()

    print(f"Total time for window update handling: {end_time - start_time:.2f} seconds")
    
    response_data = {
        "Total time for window update handling": end_time - start_time
    }
    analyze_results("Window_Update Frame Test", response_data)

await test_window_update_frame_and_analyze()

Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:25 GMT'), (b'content-type', b'text/html'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-18b4"'), (b'accept-ranges', b'bytes'), (b'content-length', b'6324'), (b'x-backend-header-rtt', b'0.002241'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600, h3-29=":443"; ma=3600'), (b'via', b'2 nghttpx'), (b'x-frame-options', b'SAMEORIGIN'), (b'x-xss-protection', b'1; mode=block'), (b'x-content-type-options', b'nosniff')]
Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:25 GMT'), (b'content-type', b'text/css'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-98aa"'), (b'accept-ranges', b'bytes'), (b'content-length', b'39082'), (b'x-backend-header-rtt', b'0.001804'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600

Total time for window update handling: 10.14 seconds
Raw Response Data: {'Total time for window update handling': 10.143510580062866}


# GoAway Frame

In [17]:
async def test_goaway_frame_and_analyze():
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)

    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())

    start_time = time.time()

    # Send a GOAWAY frame
    conn.close_connection()
    writer.write(conn.data_to_send())
    await writer.drain()

    # Wait briefly to allow any pending frames to be processed
    await asyncio.sleep(0.5)

    end_time = time.time()

    writer.close()
    await writer.wait_closed()

    print(f"Total time until connection termination: {end_time - start_time:.2f} seconds")
    
    response_data = {
        "Total time until connection termination": end_time - start_time
    }
    analyze_results("GoAway Frame Test", response_data)

await test_goaway_frame_and_analyze()

Total time until connection termination: 0.50 seconds
Raw Response Data: {'Total time until connection termination': 0.4985079765319824}


# A sequence of HEADERS, SETTINGS, and DATA frames is sent to the server

In [18]:

async def test_combined_frames_and_analyze():
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443
    path = '/'  

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)

    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())

    # Send HEADERS frame
    headers = [
        (':method', 'GET'),
        (':authority', host),
        (':scheme', 'https'),
        (':path', path),
    ]
    stream_id = conn.get_next_available_stream_id()
    conn.send_headers(stream_id, headers)
    writer.write(conn.data_to_send())

    # Send SETTINGS frame with correct setting reference
    new_settings = {
        SettingCodes.MAX_CONCURRENT_STREAMS: 10,  
    }
    conn.update_settings(new_settings)
    writer.write(conn.data_to_send())

    # Send DATA frame
    data_payload = b'Hello, world!'  
    conn.send_data(stream_id, data_payload, end_stream=True)
    writer.write(conn.data_to_send())
    await writer.drain()

    start_time = time.time()
    data_received = 0

    while True:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                data_received += len(event.data)
                print('Chunk of response data received:', len(event.data))
            elif isinstance(event, StreamEnded):
                print('Stream ended')
                break
            elif isinstance(event, SettingsAcknowledged):
                print('Settings acknowledged by server')

    end_time = time.time()

    writer.close()
    await writer.wait_closed()

    print(f"Total response time: {end_time - start_time:.2f} seconds")
    print(f"Total data received: {data_received} bytes")
    
    response_data = {
        "Total response time": end_time - start_time,
        
    }
    analyze_results("Frame Combination Test Header, Setting, Data", response_data)


await test_combined_frames_and_analyze()

Settings acknowledged by server
Settings acknowledged by server
Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:37 GMT'), (b'content-type', b'text/html'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-18b4"'), (b'accept-ranges', b'bytes'), (b'content-length', b'6324'), (b'x-backend-header-rtt', b'0.002272'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600, h3-29=":443"; ma=3600'), (b'via', b'2 nghttpx'), (b'x-frame-options', b'SAMEORIGIN'), (b'x-xss-protection', b'1; mode=block'), (b'x-content-type-options', b'nosniff')]
Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:37 GMT'), (b'content-type', b'text/css'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-98aa"'), (b'accept-ranges', b'bytes'), (b'content-length', b'39082'), (b'x-backend-header-rtt', b'0.001135'), (b'strict-transport-security', b'max-age=315360

# Headers and Data Frames

In [19]:

async def test_frame_combination(frame_sequence):
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443
    path = '/'  

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)
    
    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())

    start_time = time.time()  

    stream_id = conn.get_next_available_stream_id()

    for frame in frame_sequence:
        if frame == "HEADERS":
            headers = [
                (':method', 'GET'),
                (':authority', host),
                (':scheme', 'https'),
                (':path', path),
            ]
            conn.send_headers(stream_id, headers)
        elif frame == "DATA":
            data = b'Hello, world!'
            conn.send_data(stream_id, data, end_stream=True)
    

        writer.write(conn.data_to_send())
        await writer.drain()

    while True:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                print('Response data:', event.data)
            elif isinstance(event, StreamEnded):
                print('Stream ended')
                break

    end_time = time.time()  # Record the end time

    writer.close()
    await writer.wait_closed()
    
    response_data = {
        "Total response time": end_time - start_time
       
    }
    analyze_results("Frame Combination Test Header, Data", response_data)

# Test with HEADERS and DATA frame combination
await test_frame_combination(["HEADERS", "DATA"])

Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:48 GMT'), (b'content-type', b'text/html'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-18b4"'), (b'accept-ranges', b'bytes'), (b'content-length', b'6324'), (b'x-backend-header-rtt', b'0.003321'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600, h3-29=":443"; ma=3600'), (b'via', b'2 nghttpx'), (b'x-frame-options', b'SAMEORIGIN'), (b'x-xss-protection', b'1; mode=block'), (b'x-content-type-options', b'nosniff')]
Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:48 GMT'), (b'content-type', b'text/css'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-98aa"'), (b'accept-ranges', b'bytes'), (b'content-length', b'39082'), (b'x-backend-header-rtt', b'0.001499'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600

Raw Response Data: {'Total response time': 10.14429235458374}


# Header, Settings and Window_Update

In [20]:

async def test_frame_combination(frame_sequence):
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443
    path = '/'  

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)
    
    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())
    start_time = time.time()
    stream_id = conn.get_next_available_stream_id()

    for frame in frame_sequence:
        if frame == "HEADERS":
            headers = [
                (':method', 'GET'),
                (':authority', host),
                (':scheme', 'https'),
                (':path', path),
            ]
            conn.send_headers(stream_id, headers)
        elif frame == "SETTINGS":
            new_settings = {
                SettingCodes.INITIAL_WINDOW_SIZE: 65535
            }
            conn.update_settings(new_settings)
        elif frame == "WINDOW_UPDATE":
            conn.increment_flow_control_window(65535)
        

        writer.write(conn.data_to_send())
        await writer.drain()

    while True:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                print('Response data:', event.data)
            elif isinstance(event, StreamEnded):
                print('Stream ended')
            elif isinstance(event, WindowUpdated):
                print('Window update frame received')

    writer.close()
    end_time = time.time()
    await writer.wait_closed()
    
    response_data = {
        "Total response time": end_time - start_time,
        
    }
    analyze_results("Frame Combination Test Header, Settings, Windows_Update", response_data)


# Test with HEADERS, SETTINGS, and WINDOW_UPDATE frame combination
await test_frame_combination(["HEADERS", "SETTINGS", "WINDOW_UPDATE"])

Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:30:59 GMT'), (b'content-type', b'text/css'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-98aa"'), (b'accept-ranges', b'bytes'), (b'content-length', b'39082'), (b'x-backend-header-rtt', b'0.001384'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600, h3-29=":443"; ma=3600'), (b'via', b'2 nghttpx'), (b'x-frame-options', b'SAMEORIGIN'), (b'x-xss-protection', b'1; mode=block'), (b'x-content-type-options', b'nosniff'), (b'x-http2-push', b'1')]
Response data: b'html,body,div,span,applet,object,iframe,h1,h2,h3,h4,h5,h6,p,blockquote,pre,a,abbr,acronym,address,big,cite,code,del,dfn,em,img,ins,kbd,q,s,samp,small,strike,strong,sub,sup,tt,var,b,u,i,center,dl,dt,dd,ol,ul,li,fieldset,form,label,legend,table,caption,tbody,tfoot,thead,tr,th,td,article,aside,canvas,details,embed,figure,figcaption,footer,header,hgroup,menu,nav,output,ruby,

Raw Response Data: {'Total response time': 10.143590927124023}


# Header, Priority and Ping

In [21]:

async def test_frame_combination(frame_sequence):
    ssl_context = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
    ssl_context.set_alpn_protocols(["h2"])

    host = 'nghttp2.org'
    port = 443
    path = '/'  

    reader, writer = await asyncio.open_connection(host, port, ssl=ssl_context)
    
    config = H2Configuration(client_side=True)
    conn = H2Connection(config=config)
    conn.initiate_connection()
    writer.write(conn.data_to_send())
    start_time = time.time()
    stream_id = conn.get_next_available_stream_id()

    for frame in frame_sequence:
        if frame == "HEADERS":
            headers = [
                (':method', 'GET'),
                (':authority', host),
                (':scheme', 'https'),
                (':path', path),
            ]
            conn.send_headers(stream_id, headers)
        elif frame == "PRIORITY":
            try:
                conn.prioritize(stream_id, weight=256, depends_on=0, exclusive=False)
            except ProtocolError as e:
                print(f"Protocol Error: {e}")
        elif frame == "PING":
            conn.ping(b'12345678')
        
        writer.write(conn.data_to_send())
        await writer.drain()

    while True:
        data = await reader.read(4096)
        if not data:
            break

        events = conn.receive_data(data)
        for event in events:
            if isinstance(event, ResponseReceived):
                print('Response headers:', event.headers)
            elif isinstance(event, DataReceived):
                print('Response data:', event.data)
            elif isinstance(event, StreamEnded):
                print('Stream ended')
            elif isinstance(event, PingReceived):
                print('Ping received')
                
    end_time = time.time()
    writer.close()
    await writer.wait_closed()
    
    response_data = {
        "Total response time": end_time - start_time,
        
    }
    analyze_results("Frame Combination Test Header, Priority, Ping", response_data)


# Test with HEADERS, PRIORITY, and PING frame combination
await test_frame_combination(["HEADERS", "PRIORITY", "PING"])

Response headers: [(b':status', b'200'), (b'date', b'Mon, 11 Dec 2023 02:31:10 GMT'), (b'content-type', b'text/css'), (b'last-modified', b'Fri, 27 Oct 2023 13:41:30 GMT'), (b'etag', b'"653bbe0a-98aa"'), (b'accept-ranges', b'bytes'), (b'content-length', b'39082'), (b'x-backend-header-rtt', b'0.001436'), (b'strict-transport-security', b'max-age=31536000'), (b'server', b'nghttpx'), (b'alt-svc', b'h3=":443"; ma=3600, h3-29=":443"; ma=3600'), (b'via', b'2 nghttpx'), (b'x-frame-options', b'SAMEORIGIN'), (b'x-xss-protection', b'1; mode=block'), (b'x-content-type-options', b'nosniff'), (b'x-http2-push', b'1')]
Response data: b'html,body,div,span,applet,object,iframe,h1,h2,h3,h4,h5,h6,p,blockquote,pre,a,abbr,acronym,address,big,cite,code,del,dfn,em,img,ins,kbd,q,s,samp,small,strike,strong,sub,sup,tt,var,b,u,i,center,dl,dt,dd,ol,ul,li,fieldset,form,label,legend,table,caption,tbody,tfoot,thead,tr,th,td,article,aside,canvas,details,embed,figure,figcaption,footer,header,hgroup,menu,nav,output,ruby,

Raw Response Data: {'Total response time': 10.14760971069336}


In [22]:
# Printing each test result
for result in analysis_results:
    print(f"\nResults for {result['Test Name']}:")
    for key, value in result['Response'].items():
        print(f"{key}: {value}")



Results for Headers Frame Test:
Total response time: 10.468260765075684
Total data received: 45406

Results for Data Frame Test:
Total response time: 10.146790742874146
Total data received: 0

Results for Settings Frame Test:
Total time for settings acknowledgment: 0.28667449951171875

Results for Window_Update Frame Test:
Total time for window update handling: 10.143510580062866

Results for GoAway Frame Test:
Total time until connection termination: 0.4985079765319824

Results for Frame Combination Test Header, Setting, Data:
Total response time: 10.152052402496338

Results for Frame Combination Test Header, Data:
Total response time: 10.14429235458374

Results for Frame Combination Test Header, Settings, Windows_Update:
Total response time: 10.143590927124023

Results for Frame Combination Test Header, Priority, Ping:
Total response time: 10.14760971069336


In [23]:
# Convert analysis_results to DataFrame
df_results = pd.DataFrame(analysis_results)

In [24]:
# Extract specific metrics into separate columns
for metric in ['Total response time', 'Total data received', 'Total time for settings acknowledgment', 
               'Total time for window update handling', 'Total time until connection termination']:
    df_results[metric] = df_results['Response'].apply(lambda x: x.get(metric, 0))

# Display the DataFrame for verification
print(df_results)

                                           Test Name  \
0                                 Headers Frame Test   
1                                    Data Frame Test   
2                                Settings Frame Test   
3                           Window_Update Frame Test   
4                                  GoAway Frame Test   
5       Frame Combination Test Header, Setting, Data   
6                Frame Combination Test Header, Data   
7  Frame Combination Test Header, Settings, Windo...   
8      Frame Combination Test Header, Priority, Ping   

                                            Response  Total response time  \
0  {'Total response time': 10.468260765075684, 'T...            10.468261   
1  {'Total response time': 10.146790742874146, 'T...            10.146791   
2  {'Total time for settings acknowledgment': 0.2...             0.000000   
3  {'Total time for window update handling': 10.1...             0.000000   
4  {'Total time until connection termination': 0....  